In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
key_path = "./creds/service_acc_auth.json"
credentials = service_account.Credentials.from_service_account_file(key_path, scopes = ["https://www.googleapis.com/auth/cloud-platform"])
bigquery_client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [2]:
# query new data <= 1/1
union_weeks = """
SELECT * , 
FROM `scanning-database.scanning_data_backup.W01`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W02`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W03`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W04`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W05`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W06`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W07`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W08`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W09`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W10`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W11`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W12`
"""

In [3]:
# Overwriting old PowerBI source data
job_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10,
destination = 'scanning-database.PowerBI.PowerBI',
write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
) # set query limit to 10G
query_job = bigquery_client.query(union_weeks, job_config)  # API request
rows = query_job.result()  # Waits for query to finish
data_in_use = rows.to_dataframe()

In [4]:
data_in_use.sort_values(by=['Week']).tail()

,Start_Time,Finish_Time,Activity,Seq_Code,Recipe_Name,Break_Reasons,Missing_Ingredients,Kitting_Line,Assembly_Batch,Event_Shift,Team_Leader,Pickers_Count,Time_Consumption,Week
34870,2021-03-18 20:00:11+00:00,2021-03-18 20:03:43+00:00,Production,0016 of 0043,N4,None,None,KL14,Sat-NI Delivery,Thursday Afternoon,iXUsr_Fuataimi(10462),4,3.53,2021-12
34869,2021-03-18 19:05:36+00:00,2021-03-18 19:09:42+00:00,Production,0008 of 0043,N4,None,None,KL14,Sat-NI Delivery,Thursday Afternoon,iXUsr_Fuataimi(10462),4,4.10,2021-12
34868,2021-03-19 15:04:52+00:00,2021-03-19 15:08:19+00:00,Production,0042 of 0062,B2,None,None,KL13,Sat-NI Delivery,Friday Afternoon,iXUsr_Wang(9965),4,3.45,2021-12
34866,2021-03-19 12:54:26+00:00,2021-03-19 12:57:49+00:00,Production,0028 of 0028,H2,None,None,KL9,Sat-NI Delivery,Friday Morning,iXUsr_Pontoh(6763),4,3.38,2021-12
20437,2021-03-18 15:52:41+00:00,2021-03-19 06:11:57+00:00,Factory Closed,None,I4,None,None,KL5,None,Friday Morning,iXUsr_Gwen(7292),5,859.27,2021-12


In [5]:
data_in_use.shape

(73122, 14)